# Fallstudie

## Import Pakete

In [241]:
import pandas as pd
import numpy as np
import sqlite3
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sqlalchemy import create_engine 
from sklearn.neighbors import KNeighborsClassifier
import pickle

## Import Datenbank

In [242]:

cnx = create_engine('sqlite:///C:/Users/Kilian/Notebook/VisualStudio_projekte/Fallstudie/Kundendaten.db').connect() 

  
df = pd.read_sql_table('testdaten', cnx) 

In [243]:
#df = df.head(100000)

## Datenmodellierung 

In [244]:
df = df.drop(["Datum"], 1)
df = df.drop(["Anzahl"], 1)
df = df.drop(["Gewinn"], 1)
df = df.drop(["Jahr"], 1)
df = df.drop(["Monat"], 1)
df = df.drop(["Tag"], 1)
df = df.drop(["index"], 1)

In [245]:
df.head()

,Alter,Geschlecht,Job,Familienstand,Kinder,Gehalt,Angebotenes Produkt,Gekauft
0,47,M,Öffentlicher Dienst,verheiratet,ja,42000.0,Depotkonto,nein
1,45,M,Ingenieurswesen,verheiratet,ja,76000.0,Bausparvertrag,ja
2,23,W,Handwerk,ledig,nein,23000.0,Kredit,nein
3,60,W,Administrativ,verheiratet,ja,57000.0,Kredit,nein
4,60,M,Informatik,aufgelöste Beziehung,ja,123000.0,Kredit,nein


In [246]:
pd.options.mode.chained_assignment = None  # default='warn'
df["Altersgruppe"] = ">65"

df["Altersgruppe"][df["Alter"] > 65] = ">65"
df["Altersgruppe"][(df["Alter"] >= 50) & (df["Alter"] < 65)] = "50 - 65"
df["Altersgruppe"][(df["Alter"] >= 30) & (df["Alter"] < 50)] = "30 - 49"
df["Altersgruppe"][(df["Alter"] >= 18) & (df["Alter"] < 30)] = "18 - 29"
df["Altersgruppe"][df["Alter"] < 18] = "<18"

In [247]:
df["balance_cat"] = "> 100000"

df["balance_cat"][df["Gehalt"] > 100000] = "> 100000"
df["balance_cat"][(df["Gehalt"] >= 80000) & (df["Gehalt"] < 100000)] = "80000 - 99999"
df["balance_cat"][(df["Gehalt"] >= 60000) & (df["Gehalt"] < 80000)] = "60000 - 79999"
df["balance_cat"][(df["Gehalt"] >= 40000) & (df["Gehalt"] < 60000)] = "40000 - 59999"
df["balance_cat"][(df["Gehalt"] >= 20000) & (df["Gehalt"] < 40000)] = "20000 - 39999"
df["balance_cat"][(df["Gehalt"] >= 0) & (df["Gehalt"] < 20000)] = "0 - 19999"
df["balance_cat"][df["Gehalt"] < 0] = "< 0"

In [248]:
df["Produkt"] = df["Angebotenes Produkt"]
df["Produkt"] = df["Produkt"].replace(["Girokonto", "Kredit","Tagesgeldkonto","Depotkonto", "Altersvorsorge","Versicherung", "Bausparvertrag"],[1,2,3,4,5,6,7])
df = df.drop(["Angebotenes Produkt"], 1)

In [249]:

df["Job"] = df["Job"].replace(["Studium", "Öffentlicher Dienst", "Rente", "Informatik", "Handel", "Handwerk", "Administrativ", "Ingenieurswesen", "Management", "Arbeitslos"],[0,1,2,3,4,5,6,7,8,9])

In [250]:
df["Geschlecht"] = df["Geschlecht"].replace(["M", "W", "D"],[1,2,3])

In [251]:
df["Familienstand"] = df["Familienstand"].replace(["verheiratet", "ledig", "aufgelöste Beziehung"],[1,2,3])

In [252]:
df["Kinder"] = df["Kinder"].replace(["ja", "nein"],[1,0])

In [253]:
df["Altersgruppe"] = df["Altersgruppe"].replace(["<18", "18 - 29", "30 - 49", "50 - 65", ">65"],[0,1,2,3,4])

In [254]:
df["balance_cat"] = df["balance_cat"].replace(["< 0", "0 - 19999", "20000 - 39999", "40000 - 59999", "60000 - 79999", "80000 - 99999", "> 100000"],[1,2,3,4,5,6,7])
df = df.drop(["Gehalt"], axis=1)

In [255]:
df = df.drop(["Alter"], axis=1)

In [256]:
df["Gekauft"] = df["Gekauft"].replace(["nein", "ja"],[0, 1])

In [257]:
labels = np.array(df["Gekauft"])
#labels = np.array(df["Produkt"])

In [258]:
dfTest = df

In [259]:
df = df.drop(["Gekauft"], axis=1)
#df = df.drop(["Produkt"], axis=1)

In [260]:
df_list = list(df.columns)

## Algorithmen

### Trainings- und Testdaten vorbereiten

In [261]:
df = np.array(df)

In [262]:
train_data, test_data, train_labels, test_labels = train_test_split(df, labels, test_size = 0.25, random_state = 42)

In [263]:
print('Training Features Shape:', train_data.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_data.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (285519, 7)
Training Labels Shape: (285519,)
Testing Features Shape: (95173, 7)
Testing Labels Shape: (95173,)


### RANDOM FOREST CLASSIFIER

In [264]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

In [265]:
#rf.fit(train_data, train_labels);

In [266]:
#pred_rf = rf.predict(test_data)


In [267]:
#errors_rf = abs(pred_rf - test_labels)
#errors_rf = [x for x in errors_rf if x != 0]

In [268]:
#print(f"Accuracy RF = {(len(test_labels)-len(errors_rf))/len(test_labels)}")

In [269]:
#confusion_matrix_rf = sklearn.metrics.confusion_matrix(test_labels, pred_rf)
#tn_rf, fp_rf, fn_rf, tp_rf = confusion_matrix_rf.ravel()

#Precision_rf = tp_rf/(tp_rf+fp_rf)
#Recall_rf = tp_rf/(tp_rf+fn_rf)
#F1_rf = 2*((Precision_rf*Recall_rf)/(Precision_rf+Recall_rf))

#print(f"Precision RF = {Precision_rf}")
#print(f"Recall Rf = {Recall_rf}")
#print(f"F1 Score RF = {F1_rf}")

### K NEAREST NEIGHBORS

In [270]:
#classifier = KNeighborsClassifier(n_neighbors=7)

In [271]:
#classifier.fit(train_data, train_labels)

In [272]:
#pred_knn = classifier.predict(test_data)

In [273]:
#errors_knn = abs(pred_knn - test_labels)
#errors_knn = [x for x in errors_knn if x != 0]

In [274]:
#print(f"Accuracy KNN = {(len(test_labels)-len(errors_knn))/len(test_labels)}")

In [275]:
#confusion_matrix_knn = sklearn.metrics.confusion_matrix(test_labels, pred_knn)
#tn_knn, fp_knn, fn_knn, tp_knn = confusion_matrix_knn.ravel()

#Precision_knn = tp_knn/(tp_knn+fp_knn)
#Recall_knn = tp_knn/(tp_knn+fn_knn)
#F1_knn = 2*((Precision_knn*Recall_knn)/(Precision_knn+Recall_knn))

#print(f"Precision KNN = {Precision_knn}")
#print(f"Recall KNN = {Recall_knn}")
#print(f"F1 Score KNN = {F1_knn}")


### XGBoost Classifier

In [276]:
 from scipy import stats
 from scipy.stats import randint
 from sklearn.model_selection import RandomizedSearchCV
 from sklearn.metrics import precision_score,recall_score,accuracy_score,f1_score,roc_auc_score

In [277]:
model = XGBClassifier(
 objective= 'binary:logistic')

In [278]:
param_dist = {'n_estimators': stats.randint(150, 1000),
              'learning_rate': stats.uniform(0.01, 0.6),
              'subsample': stats.uniform(0.3, 0.9),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.9),
              'min_child_weight': [1, 2, 3, 4]
             }



In [279]:
clf = RandomizedSearchCV(model, 
                         param_distributions = param_dist,
                         n_iter = 5,
                         scoring = 'roc_auc', 
                         error_score = 0, 
                         verbose = 3, 
                         n_jobs = -1)

In [280]:
clf.fit(train_data, train_labels)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:   27.8s remaining:    8.7s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 12.7min finished
[18:31:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=None, error_score=0,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,...
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002AB332B78C8>,
                                        'max_depth': [3, 4, 5, 6, 7, 8, 9],
               

In [281]:
pred_xgb = clf.predict(test_data)

In [282]:
errors_xgb = abs(pred_xgb - test_labels)
errors_xgb = [x for x in errors_xgb if x != 0]

In [283]:
print(f"Accuracy XGB = {(len(test_labels)-len(errors_xgb))/len(test_labels)}")

Accuracy XGB = 0.7209502695092096


In [284]:
confusion_matrix_xgb = sklearn.metrics.confusion_matrix(test_labels, pred_xgb)
tn_xgb, fp_xgb, fn_xgb, tp_xgb = confusion_matrix_xgb.ravel()

Precision_xgb = tp_xgb/(tp_xgb+fp_xgb)
Recall_xgb = tp_xgb/(tp_xgb+fn_xgb)
F1_xgb = 2*((Precision_xgb*Recall_xgb)/(Precision_xgb+Recall_xgb))

print(f"Precision xgb = {Precision_xgb}")
print(f"Recall xgb = {Recall_xgb}")
print(f"F1 Score xgb = {F1_xgb}")


Precision xgb = 0.6007807684405178
Recall xgb = 0.20504908835904628
F1 Score xgb = 0.3057458043603283


## Funktion für Applikation

In [285]:
for x in range(1,8):
     print(model.predict_proba(np.array([[2,0,2,0,1,2,x]])).reshape((1,-1)))
        
        
def vorschlag(kunde):
    kunde = np.array(kunde).reshape((1,-1))
    
    kunde[0][6] = 1
    prob1 = model.predict_proba(kunde)[0][1]
    
    kunde[0][6] = 2
    prob2 = model.predict_proba(kunde)[0][1]
    
    kunde[0][6] = 3
    prob3 = model.predict_proba(kunde)[0][1]
    
    kunde[0][6] = 4
    prob4 = model.predict_proba(kunde)[0][1]
    
    kunde[0][6] = 5
    prob5 = model.predict_proba(kunde)[0][1]
    
    kunde[0][6] = 6
    prob6 = model.predict_proba(kunde)[0][1]
    
    kunde[0][6] = 7
    prob7 = model.predict_proba(kunde)[0][1]
    
    produkt = {"Girokonto":prob1,"Kredit":prob2, "Tagesgeldkonto":prob3, "Depotkonto":prob4, "Altersvorsorge":prob5, "Versicherung":prob6, "Bausparvertrag":prob7}
    
    print(f"Top 3 Produktvorschläge:")
    print(f"1. {sorted(produkt, key=produkt.get, reverse=True)[:3][0]} mit {round((produkt.get(sorted(produkt, key=produkt.get, reverse=True)[:3][0])*100),2)}% Erfolgschance")
    print(f"2. {sorted(produkt, key=produkt.get, reverse=True)[:3][1]} mit {round((produkt.get(sorted(produkt, key=produkt.get, reverse=True)[:3][1])*100),2)}% Erfolgschance")
    print(f"3. {sorted(produkt, key=produkt.get, reverse=True)[:3][2]} mit {round((produkt.get(sorted(produkt, key=produkt.get, reverse=True)[:3][2])*100),2)}% Erfolgschance")
    

NotFittedError: need to call fit or load_model beforehand

In [87]:
vorschlag(np.array([[2,0,2,0,1,2,x]]).reshape((1,-1)))


Top 3 Produktvorschläge:
1. Girokonto mit 39.32% Erfolgschance
2. Depotkonto mit 18.78% Erfolgschance
3. Versicherung mit 14.11% Erfolgschance


In [88]:
filename = 'jungbank_xgb.sav'
pickle.dump(model, open(filename, 'wb'))

In [89]:
dfTest.head()

,Geschlecht,Job,Familienstand,Kinder,Gekauft,Altersgruppe,balance_cat,Produkt
0,1,1,1,1,0,2,4,4
1,1,7,1,1,1,2,5,7
2,2,5,2,0,0,1,3,2
3,2,6,1,1,0,3,4,2
4,1,3,3,1,0,3,7,2
